In this tutorial we show how Twitter and clickstream data can be processed to extract articles from urls, and how we extract topics from news articles using LDA. 
We cover the following steps: 

1. Extracting links from tweets. 
2. Expanding URLs.
3. Restricting to news domains and articles.
4. Extracting text from articles. 
5. Restricting articles to those that mention Brexit. 
6. Topic models on Brexit articles. 



# Extracting links from tweets

Data privacy and Twitter terms of service do not allow us to share original Clickstream and Twitter data. We do, however, show how the data has been processed to obtain a list of urls from each source. 

We store most of our data in MongoDB, a non-relational database which is more convenient and efficient for storing tweets, articles as well as other types of documents with a rich structure. The script below assumes that the data is stored in a MongoDB collection, but it can be easily modified to deal with raw json data, and we show later an example of working with a json file. The package we are using for woking with MongoDB in Python is Pymongo. 
Our first task is to select only tweets written between 17 February and 23 June 2016 and pull out the urls, along with other useful information. 

In [9]:
import pymongo 

#Check if we can connect to the database:
try:
    connection=pymongo.MongoClient()
    print "Connected successfully!!!"
except pymongo.errors.ConnectionFailure, e:
   print "Could not connect to MongoDB: %s" % e 

Connected successfully!!!


In [11]:
#Connect to the database and select the 
#conn = pymongo.MongoClient('localhost', 27017)
db = connection['HWtwitter']
collection = db['brexit']

#How many tweets do we have in this collection?
collection.find().count()

74126534

In [12]:
# Import datetime modules for working with dates
from datetime import datetime
import pytz
# Define date range of interest
start_date = datetime(2016, 2, 17, 00, 00, 0, 0, pytz.UTC)
end_date = datetime(2016, 6, 24, 00, 00, 0, 0, pytz.UTC)
# Check if range defined correctly. Is end date after start date? 
end_date>start_date

True

In [ ]:
# Open a csv file and write out the relevant fields for the tweets that 
# are in the date range, and which include urls. 
import csv
import unicodedata
#Open a nex csv file to write to it: 
with open('brexit_links.csv', 'w') as outfile:                               
    writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    header = ['mongo_id',  'tweet_id', 'user_id', 'date_str', \
              'number_retweets', 'user_followers', 'is_retweet', \
              'orig_user_id', 'orig_number_retweets', \
              'orig_user_followers', 'expanded_url_1']    
    #write the header with the names of the \
    # variables of interest to file:
    writer.writerow(header)                                                                            
    #use the very useful Cursor method to loop through \
    #the tweets in the database. Define the cursor. 
    cursor=collection.find()     
    for i in cursor:
        #Turn tweet date field to utf-8 format \
        #to prevent character encoding errors.  
        utf8_date=unicodedata.normalize('NFKD', \
                                        i["created_at"]).encode('utf-8')                       
        # Turn date field into datetime format. 
        tweet_date=datetime.strptime(utf8_date,\
                                     '%a %b %d %H:%M:%S +0000 %Y')
        #If tweet ascreated between the dates of interest, 
        #extract the following variables of interest: 
        if (pytz.utc.localize(tweet_date)>=start_date and \
            pytz.utc.localize(tweet_date)<=end_date):    
            try:
                # Only select tweets that include urls. 
                # This works because all the other fields 
                # defined at this level are always in the data.
                # So this only fails if there is no url. 
                expanded_url_1=i["entities"]["urls"][0]["expanded_url"]
                mongo_id=i["_id"]
                tweet_id=i["id"]
                user_id=i["user"]["id_str"]
                date_str=tweet_date
                number_retweets=i["retweet_count"]
                user_followers=i["user"]["followers_count"]
                try:
                    # These fields only exist if the tweet is a retweet.
                    # Otherwise, write them out as empty. 
                    retweet_id=i["retweeted_status"]["id"]
                    orig_user_id=i["retweeted_status"]["user"]["id_str"]
                    orig_number_retweets=i["retweeted_status"]["retweet_count"]
                    orig_user_followers=i["retweeted_status"]["user"]["followers_count"]
                    is_retweet=1
                except: 
                    is_retweet=0
                    retweet_id=0
                    orig_number_retweets=0
                    orig_user_followers=0
                # Write results to csv: 
                writer.writerow([mongo_id]+[tweet_id]+[user_id]+[date_str]+\
                                [number_retweets]+[user_followers]+[is_retweet]+\
                                [orig_user_id]+[orig_number_retweets]+\
                                [orig_user_followers]+[expanded_url_1]])
            except:     
                pass

# The output is a csv file with urls and information about the 
# tweets they were extracted from. 

In [ ]:
# Read file into Pandas, remove duplicates in termd of urls, write out urls only. 
import pandas as pd
# Read in data:
df = pd.read_csv("brexit_links.csv", sep="\t")
# Drop duplicate values:
df_no_duplic=df.drop_duplicates(subset='expanded_url_1',\
                                keep="first")
# Write new data to file. 
df_no_duplic["expanded_url_1"].to_csv("brexit_unique_time_intervala.csv", \
                                      index=False)

# Resolving urls
The same url can appear in the Twitter and Clickstream data under multiple forms, depending on whether it was shortened and the method used to shorten it, or if it is a redirect from social media, etc. By resolving the url we ensure that all these different formats are reduced to a single article url. This step is necessar since we don't want to waste any resources extracting text from the same article multiple times (and then merging them back together by text). 
The script below is used to resolve urls both in the Twitter data and in the Clickstream data. 

In [ ]:
# Import required modules
import csv
import grequests
import requests
import re
import itertools
import sys
import os
import urlclean

# Define working directory and file names: 
os.chdir("F:/exponet/twitter_brexit")
F_NAME_IN = 'brexit_unique_time_interval.csv'
F_NAME_OUT = 'brexit_unique_time_interval_expanded.csv'
F_NAME_OUT_ERR = 'brexit_unique_urls_expanded_exceptions.csv'

# Handle all http formats: 
rx_http = re.compile('(?<=http)s(?=:\/\/)')

# Read in urls 
with open(F_NAME_IN,'r') as f_in:
	url_list = [row.strip('\n') for row in f_in]

# Define how exceptions are handled: 
class ExceptionHandler:
	def __init__(self):
		self.f_out = open(F_NAME_OUT_ERR,'wb')
		self.w = csv.writer(self.f_out, delimiter='\t', \
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
		
	def __enter__(self):
		return self
		
	def __exit__(self, exc_type, exc_value, traceback):
		self.f_out.close()
	
	def callback(self, request, exception):
		try:
			self.w.writerow([request.url, repr(exception.message)])
		except UnicodeError:
			print 'Following string is not UTF-8:'
			print repr(request.url), repr(exception.message)

def exception_handler(request, exception):
	pass

# Disable warnings (not a very good idea, make sure you \
# read about it before using this option)
requests.packages.urllib3.disable_warnings()

# Define the function that retries the urls from the request response 
def get_url(response):
	if response.history:
		series = list(itertools.chain(response.history,[response]))
		for i in range(len(series)):
			if series[i].status_code == 301 or \
            (i < len(series) - 1 and rx_http.sub('', series[i].url) \
             == rx_http.sub('', series[i+1].url)): pass
			else: break
		retval = [series[0].url, series[i].url if i > 0 else '', \
                  series[i].status_code]
	else: retval = [response.url, '', response.status_code]
	response.close()
	return retval

# Visit all urls in the list: 
reqs = (grequests.head(u, verify = False, timeout = 3) \
        for u in url_list)

# Write resolved urls to file: 
with open(F_NAME_OUT,'wb') as f_out, ExceptionHandler() as eh:
	w = csv.writer(f_out, delimiter='\t', quotechar='"', \
                   quoting=csv.QUOTE_MINIMAL)
	for res in grequests.imap(reqs, exception_handler = eh.callback, \
                              size = 500):
		w.writerow(get_url(res))


After resolving the urls, we need to drop dupplicates again to end up with a set of unique urls. We do this just as we did before: 

In [ ]:
# Read in data:
df = pd.read_csv("brexit_unique_time_interval_expanded.csv", sep="\t")
# Drop duplicate values:
df_no_duplic=df.drop_duplicates(subset='expanded_url_1', keep="first")
# Write new data to file. 
df_no_duplic["expanded_url_1"].to_csv("brexit_unique_expanded.csv", \
                                      index=False)

# Restricting to news domains & articles
We next restrict our data to a list of 460 major news domains only, and/or to news articles on those domains. To identify articles on news domains we take advantage of the structure of news websites, which place articles in a specific location in a database. We can therefore write regular expressions that identify articles based on that location. 

In [ ]:
#Read in list of domains and pre-coded regular expressions for the articles: 

news_domains=[]
article_regex=[]
with open("alexa_news_domains_no_duplic.csv", "r") as infile:
    reader=csv.reader(infile, delimiter="\t")
    for row in reader:
        news_domains.append(str(row[0]).strip())
        article_regex.append(str(row[1]).strip())

In [ ]:
# Filter links to articles. 
with open('brexit_news_links.csv', 'w') as outfile:    
    writer = csv.writer(outfile, delimiter='\t', \
                        lineterminator='\n')
    with open("brexit_unique_expanded.csv", "r") as infile:
        reader=csv.reader(infile, delimiter="\t")
        for row in reader:
            domain_list=[i for i in news_domains if i \
                         in str(row[0])]
            try:
                domain=domain_list[0]
            except:
                domain=""
            writer.writerow(row+[domain])

# Extracting content from article links
We extracted the title, text, author, date, and other information from the article links using Diffbot, a content-extraction service. Diffbot is free for processing up to 10.000 articles per month. You need to sign up and imput your token for the script below to work. 

In [ ]:

import diffbot
import csv
import time
import sys
import json
import os


token="your_diffbot_token_here"

# You can get up to 1 link per second with the free version, 
# so we can run multiple identical scripts at the same time. 
# We do this by dividing the data into 5 sets, and running 
# the script on each set: 
RUN = int(sys.argv[1])
N_RUNS = int(sys.argv[2])

F_NAME_OUT = 'brexit_diffbot_links_1%02d.json'%RUN
F_NAME_OUT_ERR = 'brexit_diffbot_errors_1%02d.json'%RUN

# Read in the links
all_links=[]
with open("brexit_unique_expanded.csv", "r") as infile:
        reader=csv.reader(infile)
        for row in reader:
            all_links.append(row[0])

links=all_links[RUN :: N_RUNS]

# Write the articles out as .json, one per line. 
start = time.time()
with open(F_NAME_OUT_ERR, 'wb') as errfile:
    writerr=csv.writer(errfile)
    with open(F_NAME_OUT, 'wb') as outfile:
        for link in links:
            try:
                json.dump(diffbot.article(link, \
                            token=token,\discussion=False, \
                            timeout=20000), outfile)
                outfile.write('\n')
            except:
                writerr.writerow(link)
                pass
print time.time()-start, 'seconds'

Now we need to process the .json file from Diffbot to retain only the link, title, and text.  

In [ ]:
import glob, os
thepath=glob.glob("brexit_diffbot_links_*.json")
# Read all files in the folder: 
files=[]
for i in thepath:
    base=os.path.basename(i)
    files.append(base)

# Take out the fields of interest from the json files,
# write them out to a new file. 
with open ('brexit_url_content.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    header=["link", "title", "text"]
    writer.writerow(header)
    for filename in files:
        with open(filename, 'r') as f: 
            for row in f:
                try:
                    diffbot_res = json.loads(row)
                    language=diffbot_res['objects'][0]['humanLanguage']
                    if language=="en":
                        raw_title=diffbot_res['objects'][0]['title']
                        title=" ".join(raw_title.split())
                        title=title.decode('utf-8', errors='ignore')
                        raw_text=diffbot_res['objects'][0]['text']
                        link=diffbot_res['request']['pageUrl']
                        if raw_text!="":
                            text=" ".join(raw_text.split())
                            text=text.decode('utf-8', errors='ignore')
                            writer.writerow([link]+[title]+[text])
                except:
                    pass



# Keeping only articles that mention the Brexit referendum. 

In [ ]:

# Select only articles that include brexit or referendum in title of body.

with open ('mention_brexit_eu_referendum.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    with open('brexit_url_content.csv', 'r') as infile:
        reader=csv.reader(infile, delimiter='\t')
        reader.next()
        for line in reader:
            if ("brexit" in line[1].lower() or "brexit" in line[2].lower() \
                or ("eu referendum" in line[1].lower() or \
                    "membership referendum" in line[1].lower() \
                    or "european referendum" in line[1].lower()  \
                    or "uk referendum" in line[1].lower() or \
                    "eu referendum" in line[2].lower() \
                    or "membership referendum" in line[2].lower() \
                    or "european referendum" in line[2].lower()  \
                    or "uk referendum" in line[2].lower() )):
                writer.writerow(line)


# Topic models
We use the Python package Gensim to extract topics from our corpus of Brexit articles using an LDA model. 
The first step involves cleaning and tokenizing our text data: 


In [ ]:
#Import required packages:
import re
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import *
from nltk.corpus import stopwords
from gensim import corpora, models
import gensim
import csv
csv.field_size_limit(40000000)
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english')+get_stop_words('en'))

# Turn to lower case, stem, remove stopwords and numbers. 
texts=[]
with open("mention_brexit_eu_referendum.csv", "r") as infile:
    reader=csv.reader(infile, delimiter="\t")
    for row in reader:
        text=row[3].decode('utf-8', 'ignore')
        tokens=[word.lower() for word in \
                tokenizer.tokenize(stemmer.stem(text).lower()) if \
                ((word not in stop_words) and word.isalpha())]
        texts.append(tokens)

In [ ]:
# turn tokenized documents into an id to term dictionary
dictionary = corpora.Dictionary(texts)
# discard terms that appear in less than 0.05 of documents,
# and those that appear in more than 0.75 of them. 
dictionary.filter_extremes(no_below=0.05, no_above=0.75)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print('Number of unique tokens: %d' % len(tw_dictionary))
print('Number of documents: %d' % len(tw_corpus))

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50
chunksize = 2000
passes = 30
iterations = 300
eval_every = None  

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model_1 = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
             alpha='auto', eta='auto', \
             iterations=iterations, num_topics=num_topics, \
             passes=passes, eval_every=eval_every)

with open("tw_cs_combined_topics_01.txt", "a") as outfile:
    outfile.write("\n50 topics\n")
    outfile.write('\n'.join('%s %s' % x for x in \
                            model_1.print_topics(num_topics=50, num_words=30)))

This produces a list of 50 topics and the most important words for each of them. 

We can order these topics by topic coherence: 


In [ ]:
top_topics = model_1.top_topics(corpus, num_words=30)
# Average topic coherence is the sum of topic coherences 
#of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics

with open("tw_cs_combined_topics_01.txt", "a") as outfile:
    outfile.write("\nTopic coherence\n")
    outfile.write('Average topic coherence: %.4f.' % avg_topic_coherence)
    for i in top_topics: 
        outfile.write(str(i))
        outfile.write("\n")

We can also compute the probability of each topic in a document, and average thhose out by type of data: Twitter or clickstream. 

In [ ]:
article_number=0
header=["twitter", "article_number", "topic", "probability"]
with open("tw_cs_topic_probabilities_01.csv", "w") as outfile:
    writer=csv.writer(outfile,  delimiter='\t', lineterminator='\n')
    writer.writerow(header)
    for text in texts[0:29567]: 
        twitter=0
        article_number=article_number+1
        bow = dictionary.doc2bow(text)
        topics=model_1_all.get_document_topics(bow, minimum_probability=0.000001 )
        for i in topics:
            topic=i[0]
            probability=i[1]
            writer.writerow([twitter]+[article_number]+[topic]+[probability])
    for text in texts[29567:]: 
        twitter=1
        article_number=article_number+1
        bow = dictionary.doc2bow(text)
        topics=model_1_all.get_document_topics(bow, minimum_probability=0.000001 )
        for i in topics:
            topic=i[0]
            probability=i[1]
            writer.writerow([twitter]+[article_number]+[topic]+[probability])

We can then read in this file, aggregate by data source (Twitter or clickstream) and plot the average topic probabilities. 

In [1]:
import pandas as pd
df_topics=pd.read_csv("F:/exponet/clickstream/tw_cs_topic_probabilities_01.csv",\
                      sep="\t")
df_topics.head()

,cs,article_number,topic,probability
0,0,1,0,0.001232
1,0,1,1,0.000543
2,0,1,2,0.046642
3,0,1,3,0.000719
4,0,1,4,0.000986


In [6]:
# Group and aggregate data: 
grouped=df_topics.groupby(["topic", "cs"])
aggregated=grouped.agg({"probability":["mean"]})
# Display only first 5 averaged probabilities: 
aggregated[0:5]

probability
                mean
topic cs            
0     0     0.036489
      1     0.039861
1     0     0.008339
      1     0.004901
2     0     0.008857